In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from crowdnalysis.cmdstan import StanMultinomialOptimizeConsensus, StanMultinomialEtaOptimizeConsensus
from crowdnalysis.dawid_skene import DawidSkene
import numpy as np
import json

In [26]:
smoc = StanMultinomialOptimizeConsensus()
ds = DawidSkene()

In [27]:
t = 1000
num_annotations_per_task = 5
real_tau = np.array([0.1, 0.3, 0.6])
k = len(real_tau)
real_pi = np.array([[0.5, 0.3, 0.2], [0.2, 0.6, 0.2], [0.25, 0.3, 0.45]]) 
print(real_pi)
t_C, m = smoc.sample(t, num_annotations_per_task, parameters = {'tau':real_tau, 'pi':real_pi})

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_tasks
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.sample_annotations
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


[[0.5  0.3  0.2 ]
 [0.2  0.6  0.2 ]
 [0.25 0.3  0.45]]


In [28]:
smoc.model_name

'Multinomial'

In [29]:
m

array([[  0,   0,   2],
       [  0,   0,   2],
       [  0,   0,   2],
       ...,
       [999,   0,   2],
       [999,   0,   2],
       [999,   0,   2]])

In [30]:
t_A = m[:,0]+1
w_A = m[:,1]
ann = m[:,2]+1
w = 20
#t = np.unique(t_A).shape[0]
a = m.shape[0]
for t_ in range(t):
    w_A[t_*num_annotations_per_task:(t_+1)*num_annotations_per_task] = \
        np.random.choice(w, size=num_annotations_per_task, replace=False)
#w_A += 1
#w_A = np.random.choice(w, size=10)
json_data = {'w': w,
             't': t,
             'a': a,
             'k': k,
             't_A': t_A.tolist(),
             'w_A': w_A.tolist(),
             'ann': ann.tolist()}

In [31]:
print("Max w_A:",np.max(w_A))
with open('multinomial.json', 'w') as outfile:
    json.dump(json_data, outfile, indent=2)

Max w_A: 19


In [32]:
print(a)
print(m[9980:10000], t_A[9980:])

5000
[] []


In [33]:

smetaoc = StanMultinomialEtaOptimizeConsensus()
t_C, d = sm2oc.m_fit_and_compute_consensus(m, t, k, w)
d

NameError: name 'sm2oc' is not defined

In [34]:
t_C, d = smetaoc.m_fit_and_compute_consensus(m, t, k, w)
d

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/MultinomialEta.fit_and_consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


{'tau': array([0.0790507, 0.479831 , 0.441118 ]),
 'eta': array([[1.94038e-03, 2.57322e+00],
        [9.58789e-01, 4.84955e-01],
        [4.48598e-01, 6.35498e-01]]),
 'pi': array([[0.482079 , 0.481144 , 0.0367774],
        [0.191766 , 0.50023  , 0.308004 ],
        [0.294495 , 0.244292 , 0.461213 ]]),
 'log_p_t_C': array([[-13.9056 ,  -7.5702 ,  -5.58511],
        [-13.9056 ,  -7.5702 ,  -5.58511],
        [-11.3362 ,  -6.12646,  -6.40751],
        ...,
        [-11.3382 ,  -5.16767,  -6.59441],
        [-13.9075 ,  -6.61142,  -5.77201],
        [-13.9056 ,  -7.5702 ,  -5.58511]]),
 't_C': array([[2.14030e-04, 1.20751e-01, 8.79035e-01],
        [2.14030e-04, 1.20751e-01, 8.79035e-01],
        [3.10312e-03, 5.68035e-01, 4.28862e-01],
        ...,
        [1.68267e-03, 8.05036e-01, 1.93282e-01],
        [2.04537e-04, 3.01599e-01, 6.98197e-01],
        [2.14030e-04, 1.20751e-01, 8.79035e-01]])}

In [35]:
m[:,1]=0
a = ds.m_fit_and_compute_consensus(m, t, k, 1)
a


(array([[0.77975117, 0.01160843, 0.20864039],
        [0.77975117, 0.01160843, 0.20864039],
        [0.2541598 , 0.08971229, 0.65612791],
        ...,
        [0.08294018, 0.0746759 , 0.84238392],
        [0.47831396, 0.01816356, 0.50352248],
        [0.77975117, 0.01160843, 0.20864039]]),
 {'tau': array([0.30881001, 0.21229598, 0.47889401]),
  'pi': array([[[0.3340998 , 0.20854758, 0.45735262],
          [0.33733745, 0.53622389, 0.12643866],
          [0.1770161 , 0.43448655, 0.38849734]]])})

In [36]:
d = smetaoc.m_fit(m, t, k, w, t_C)
d

ValueError: no such file /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/MultinomialEta.fit.stan

In [37]:
d2 = smoc.m_compute_consensus(m, t, k, w, data=d)
d2

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/cerquide/prj/crowdnalysis/src/crowdnalysis/cmdstan/Multinomial.consensus
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


(array([[2.14031e-04, 1.20751e-01, 8.79035e-01],
        [2.14031e-04, 1.20751e-01, 8.79035e-01],
        [3.10312e-03, 5.68035e-01, 4.28862e-01],
        ...,
        [1.68267e-03, 8.05036e-01, 1.93282e-01],
        [2.04538e-04, 3.01599e-01, 6.98197e-01],
        [2.14031e-04, 1.20751e-01, 8.79035e-01]]),
 {'algorithm': 'LBFGS', 'init_alpha': 0.01, 'output_dir': '.'})